# Auto Neural Network Tuning: Boston Housing Regression


## Libreary Imoprts

In [1]:
from tensorflow.keras import models, layers
from sklearn.preprocessing import StandardScaler
import tensorflow
from numpy.random import seed
from kerastuner import HyperModel, RandomSearch, Hyperband, BayesianOptimization
from tensorflow.keras.datasets import boston_housing
import os
import tensorflow as tf  # TF > 2.0
import kerastuner as kt

print(tf.__version__)
print(kt.__version__)

seed(42)
tensorflow.random.set_seed(42)


2.3.1
1.0.2


## Loading and preparing the data

In [2]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)


## NN Model without HPT

In [3]:
model = models.Sequential()

model.add(layers.Dense(8, activation='relu', input_shape=(x_train.shape[1],),
                       kernel_initializer='random_normal', bias_initializer='zeros'))

model.add(layers.Dense(16, activation='relu', kernel_initializer='random_normal',
                       bias_initializer='zeros'))

model.add(layers.Dropout(0.1))

model.add(layers.Dense(1))

model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])

history = model.fit(x_train_scaled, y_train, validation_split=0.2, epochs=15)
mse = model.evaluate(x_test_scaled, y_test)[1]
print('MSE without tuning: {}'.format(mse))


Epoch 1/15
11/11 [==============================] - ETA: 0s - loss: 520.1207 - mse: 520.120 - 0s 14ms/step - loss: 570.3118 - mse: 570.3118 - val_loss: 639.6743 - val_mse: 639.6743
Epoch 2/15
11/11 [==============================] - ETA: 0s - loss: 583.2085 - mse: 583.208 - 0s 3ms/step - loss: 566.9042 - mse: 566.9042 - val_loss: 635.9645 - val_mse: 635.9645
Epoch 3/15
11/11 [==============================] - ETA: 0s - loss: 634.7383 - mse: 634.738 - 0s 2ms/step - loss: 563.0008 - mse: 563.0008 - val_loss: 631.5813 - val_mse: 631.5813
Epoch 4/15
11/11 [==============================] - ETA: 0s - loss: 595.9271 - mse: 595.927 - 0s 3ms/step - loss: 558.3314 - mse: 558.3314 - val_loss: 625.6187 - val_mse: 625.6187
Epoch 5/15
11/11 [==============================] - ETA: 0s - loss: 578.6592 - mse: 578.659 - 0s 3ms/step - loss: 552.7308 - mse: 552.7308 - val_loss: 619.0966 - val_mse: 619.0966
Epoch 6/15
11/11 [==============================] - ETA: 0s - loss: 560.5142 - mse: 560.514 - 0s 3m

## Model with HPT

#### Building the Hypermodel

In [4]:
class RegressionHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = models.Sequential()
        model.add(
            layers.Dense(
                units=hp.Int('units', 8, 64, 4, default=8),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                input_shape=self.input_shape,
                kernel_initializer='random_normal', bias_initializer='zeros'
            )
        )

        model.add(
            layers.Dense(
                units=hp.Int('units', 16, 64, 4, default=16),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                kernel_initializer='random_normal', bias_initializer='zeros'
            )
        )

        model.add(
            layers.Dropout(
                hp.Float(
                    'dropout',
                    min_value=0.0,
                    max_value=0.1,
                    default=0.005,
                    step=0.01)
            )
        )

        model.add(layers.Dense(1, kernel_initializer='random_normal', bias_initializer='zeros'))

        model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])

        return model

#### Initialize hypermodel

In [5]:
input_shape = (x_train.shape[1],)
hypermodel = RegressionHyperModel(input_shape)

#### Build, Run and Evaluate Random Search Tuner

In [6]:
tuner_rs = RandomSearch(
    hypermodel,
    objective='mse',
    seed=42,
    max_trials=20,
    executions_per_trial=2, overwrite=True, directory=os.path.normpath('D:\\KerasTuner\\Random')
)

tuner_rs.search(x_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=0)

best_model_rs = tuner_rs.get_best_models(num_models=1)[0]
mse_rs = best_model_rs.evaluate(x_test_scaled, y_test)[1]
print('Random search MSE: ', mse_rs)

INFO:tensorflow:Oracle triggered exit
4/4 [==============================] - ETA: 0s - loss: 67.2817 - mse: 67.281 - 0s 2ms/step - loss: 82.8706 - mse: 82.8706
Random search MSE:  82.87055969238281


#### Build, Run and Evaluate Hyperband Tuner

In [7]:
tuner_hb = Hyperband(
    hypermodel,
    max_epochs=5,
    objective='mse',
    seed=42,
    executions_per_trial=2, directory=os.path.normpath('D:\\KerasTuner\\Hyperband')
)

tuner_hb.search(x_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=0)

best_model_hb = tuner_hb.get_best_models(num_models=1)[0]
mse_hb = best_model_hb.evaluate(x_test_scaled, y_test)[1]
print('Hyperband MSE: ', mse_hb)

INFO:tensorflow:Reloading Oracle from existing project D:\KerasTuner\Hyperband\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from D:\KerasTuner\Hyperband\untitled_project\tuner0.json
INFO:tensorflow:Oracle triggered exit
4/4 [==============================] - ETA: 0s - loss: 416.2702 - mse: 416.270 - 0s 2ms/step - loss: 509.7245 - mse: 509.7245
Hyperband MSE:  509.7244873046875


#### Build, Run and Evaluate Bayesian Optimization Tuner

In [8]:
tuner_bo = BayesianOptimization(
    hypermodel,
    objective='mse',
    max_trials=10,
    seed=42,
    executions_per_trial=2,
    directory=os.path.normpath('D:\\KerasTuner\\Bayesian')
)

tuner_bo.search(x_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=1)

best_model_bo = tuner_bo.get_best_models(num_models=1)[0]
mse_bo = best_model_bo.evaluate(x_test_scaled, y_test)[1]
print('Bayesian Optimization MSE: ', mse_bo)

INFO:tensorflow:Reloading Oracle from existing project D:\KerasTuner\Bayesian\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from D:\KerasTuner\Bayesian\untitled_project\tuner0.json
INFO:tensorflow:Oracle triggered exit
4/4 [==============================] - ETA: 0s - loss: 65.1297 - mse: 65.129 - 0s 1ms/step - loss: 83.1789 - mse: 83.1789
Bayesian Optimization MSE:  83.17890167236328


#### Model Summary Comparision

In [9]:
# Old model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 112       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                144       
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 273
Trainable params: 273
Non-trainable params: 0
_________________________________________________________________


In [10]:
# New Model Summary - Random Search
best_model_rs.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 60)                840       
_________________________________________________________________
dense_1 (Dense)              (None, 60)                3660      
_________________________________________________________________
dropout (Dropout)            (None, 60)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 61        
Total params: 4,561
Trainable params: 4,561
Non-trainable params: 0
_________________________________________________________________


In [11]:
# New Model Summary - Hyperband
best_model_hb.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 60)                840       
_________________________________________________________________
dense_1 (Dense)              (None, 60)                3660      
_________________________________________________________________
dropout (Dropout)            (None, 60)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 61        
Total params: 4,561
Trainable params: 4,561
Non-trainable params: 0
_________________________________________________________________


In [12]:
# New Model Summary - BayesianOptimization
best_model_bo.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                896       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________


## References


[HyperParameters](https://keras-team.github.io/keras-tuner/documentation/hyperparameters/)  
[Tuners](https://keras-team.github.io/keras-tuner/documentation/tuners/)  
[Layer weight initializers](https://keras.io/api/layers/initializers/#randomnormal)